In [1]:
import allel
import matplotlib.pyplot as plt
import numpy as np
from dask_kubernetes import KubeCluster
from dask.distributed import Client
import functools

In [2]:
cluster = KubeCluster(n_workers=50, env={'EXTRA_PIP_PACKAGES': 'malariagen-data'})
client = Client(cluster)
client

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:    tcp://10.32.46.2:39271
distributed.scheduler - INFO -   dashboard at:                     :8787
distributed.scheduler - INFO - Receive client connection: Client-11730db8-5586-11eb-8410-8221952a1a5d
distributed.core - INFO - Starting established connection


Client Scheduler: tcp://10.32.46.2:39271 Dashboard: /user/alimanfoo@googlemail.com/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


distributed.scheduler - INFO - Register tcp://10.32.81.8:41639
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.81.8:41639
distributed.core - INFO - Starting established connection


In [3]:
!pip install -q --upgrade --no-cache-dir malariagen-data

distributed.scheduler - INFO - Register tcp://10.32.140.2:33861
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.140.2:33861
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.149.2:46773
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.149.2:46773
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.152.2:40767
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.152.2:40767
distributed.core - INFO - Starting established connection


In [4]:
import malariagen_data

In [5]:
ag3 = malariagen_data.Ag3("gs://vo_agam_release/")

In [6]:
df_samples = ag3.sample_metadata()
df_samples

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,sample_set,release,aim_fraction_colu,aim_fraction_arab,species_gambcolu_arabiensis,species_gambiae_coluzzii,species
0,AR0047-C,LUA047,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.945,0.001,gamb_colu,coluzzii,coluzzii
1,AR0049-C,LUA049,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.933,0.001,gamb_colu,coluzzii,coluzzii
2,AR0051-C,LUA051,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.937,0.002,gamb_colu,coluzzii,coluzzii
3,AR0061-C,LUA061,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.938,0.002,gamb_colu,coluzzii,coluzzii
4,AR0078-C,LUA078,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.926,0.001,gamb_colu,coluzzii,coluzzii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779,AC0295-C,K92,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,v3,0.026,0.002,gamb_colu,gambiae,gambiae
2780,AC0296-C,K93,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,v3,0.029,0.003,gamb_colu,gambiae,gambiae
2781,AC0297-C,K94,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,v3,0.026,0.002,gamb_colu,gambiae,gambiae
2782,AC0298-C,K95,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,v3,0.029,0.002,gamb_colu,gambiae,gambiae


In [7]:
gt = ag3.snp_genotypes(seq_id='X', cohort='AG1000G-BF-A')
gt

,Array,Chunk
Bytes,8.47 GB,30.00 MB
Shape,"(23385349, 181, 2)","(300000, 50, 2)"
Count,313 Tasks,312 Chunks
Type,int8,numpy.ndarray


In [9]:
gt.sum().compute()

-355782169

In [10]:
@functools.lru_cache(maxsize=None)
def compute_ac(seq_id):
    gt = ag3.snp_genotypes(seq_id=seq_id)
    gt = allel.GenotypeDaskArray(gt)
    ac = gt.count_alleles(max_allele=3).compute()
    return ac

In [38]:
def compute_allelism(mask=None):
    ret = None
    for seq_id in '2R', '2L', '3R', '3L', 'X':
        ac = compute_ac(seq_id=seq_id)
        if mask:
            filter_pass = ag3.site_filters(seq_id=seq_id, mask=mask).compute()
            ac = np.compress(filter_pass, ac, axis=0)
        n = np.bincount(ac.allelism())
        if ret is None:
            ret = n
        else:
            ret += n
    n_seg = ret[2:].sum()
    n_bi = ret[2]
    n_mu = ret[3:].sum()
    print(f"segregating: {n_seg:,}; biallelic: {n_bi:,} ({n_bi*100/n_seg:.0f}%); multiallelic: {n_mu:,} ({n_mu*100/n_seg:.0f}%)")


In [39]:
compute_allelism()

segregating: 161,639,233; biallelic: 84,416,499 (52%); multiallelic: 77,222,734 (48%)


In [40]:
compute_allelism(mask="gamb_colu_arab")

distributed.scheduler - INFO - Register tcp://10.32.2.3:33633
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.2.3:33633
distributed.core - INFO - Starting established connection


segregating: 95,071,535; biallelic: 58,474,145 (62%); multiallelic: 36,597,390 (38%)


distributed.scheduler - INFO - Retire worker names (21,)
distributed.scheduler - INFO - Retire workers {<Worker 'tcp://10.32.2.3:33633', memory: 0, processing: 0>}
distributed.scheduler - INFO - Closing worker tcp://10.32.2.3:33633
distributed.scheduler - INFO - Remove worker tcp://10.32.2.3:33633
distributed.core - INFO - Removing comms to tcp://10.32.2.3:33633
distributed.scheduler - INFO - Lost all workers
distributed.deploy.adaptive - INFO - Retiring workers [21]


In [41]:
cluster.scale(0)